<a href="https://colab.research.google.com/github/karasu1982/POS_Data_Analytics/blob/main/notebook/%E3%83%87%E3%82%B7%E3%83%AB%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# デシル分析

デシル分析は、顧客の一定期間の購買金額に基づいてグループ分けする手法の１つで、デシル（10等分）の名前の通り、購買金額が最も多いグループから最も少ないグループまで、人数が（ほぼ）同じになるように10個に分けていきます。

顧客ごとの一定期間（１か月など）の購買金額を計算する必要があるため、POSデータではなくID-POSデータが必須の分析手法です。

また、次のようにインバウンドや新型コロナなどの情勢の変化で、一定期間の購買金額だけで見ていくこと以外も求められるようになっています。


* インバウンド：ID-POSを導入しているチェーンでも、外国人観光客の爆買いによって、最も購入している顧客はポイントカード（ID）を持たない外国人というケースが出てきて、デシルが意味をなさなくなった

* 新型コロナ：顧客が遠くの店舗まで足を伸ばさなく（せなく）なったことにより、店舗へのロイヤリティが売上に大きく影響。単純な購買金額の多寡だけでなく、どの程度の頻度で来店しているかなど、それ以外の要素も重要になった


それでも、まずはデシル１の顧客（最も購入している顧客群）が、月間でいくら購入しているのかなどの基礎情報として把握するために必要な分析です。

## 環境設定

In [ ]:
%%bash
pip install duckdb-engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.4 MB/s eta 0:00:00


In [ ]:
# 標準ライブラリ
import pandas as pd
import numpy as np

# データ見える化
import seaborn as sns
import matplotlib.pyplot as plt

# DuckDB
import duckdb

## データ準備

ID-POSのサンプルデータとして、下記を利用

https://www.kyoritsu-pub.co.jp/book/b10003634.html

In [ ]:
%%bash
wget https://kyoritsu-pub.sakura.ne.jp/app/file/goods_contents/2319.zip
unzip /content/2319.zip
unzip /content/DataProcess.20151001/src/data/Tafeng/Tafeng.zip

In [ ]:
df = pd.read_csv("/content/Tafeng_dataset/Tafeng.csv")

# データ型を
df = df.astype({'CustID': 'object', 'ProductSubClass': 'object', 'ProductID': 'object'})

In [ ]:
df.head(3)

,Time,CustID,Age,Area,ProductSubClass,ProductID,Amount,Asset,SalesPrice
0,2000-11-01 00:00:00,46855,D,E,110411,4710085120468,3,51,57
1,2000-11-01 00:00:00,539166,E,E,130315,4714981010038,2,56,48
2,2000-11-01 00:00:00,663373,F,E,110217,4710265847666,1,180,135


## データ集計

期間を指定して、顧客別の総購買金額を出します。

その上で、NTILE(n) OVER(ORDER BY val DESC)という関数を用いることで、valの値が大きい順（小さい順の場合は、DESCを除く）に、n個に分割したランクをつけることができます。




In [ ]:
import duckdb

# SQLに慣れている方にとっては、少々くどいかもしれませんが、可読性を高めるために、処理を１つ１つ分解してWITH句で実行しています。
#
# 私がBigQueryの文法に慣れていることもあり、次の3点で修正が必要でした。他にも微妙な差はありそうなので、都度都度直す必要がありますね。
# ・コメントアウトは#は使わず、-- で行う
# ・文字列はダブルコーテーション（ " )ではなく、シングルクオーテーション（ ' )でくくる
# ・日付項目は、文字列を自動で日付型化しないため、DATETIME '2000-11-01'のように日付型を明示的に指定する

# まずは日次で集計してみましょう

df_output = duckdb.query(
    f"""
    WITH t AS(
      SELECT CustID, SUM(Amount * SalesPrice) AS Total_of_Sales
      FROM df
      WHERE Time BETWEEN DATETIME '2000-11-01' AND DATETIME '2000-11-30'
      GROUP BY CustID
    )
    SELECT
      CustID, Total_of_Sales,
      NTILE(10) OVER(ORDER BY Total_of_Sales DESC) AS Sales_Decile
    FROM t
    ORDER BY Total_of_Sales DESC
    """
).to_df()

作成したテーブルのアウトプット
*   CustID：顧客コード。元々あって今回集計キーにした項目
*   Total_of_Sales：総売上高
*   Sales_Decile：総売上高に基づいたデシル

総売上高が高いにソートしています。

In [ ]:
 # データ確認
df_output

,CustID,Total_of_Sales,Sales_Decile
0,1970777,14802928.0,1
1,1515749,6667922.0,1
2,1558418,6517143.0,1
3,2119083,6261437.0,1
4,2131269,6241638.0,1
...,...,...,...
16755,1913224,9.0,10
16756,2112435,9.0,10
16757,1495300,9.0,10
16758,2072784,9.0,10


In [ ]:
# 各デシルの基準となる、最大金額と最小金額
n_decile = 1

print("デシル{}の最大値:".format(n_decile), df_output[df_output["Sales_Decile"]==n_decile]["Total_of_Sales"].max())
print("デシル{}の最小値:".format(n_decile), df_output[df_output["Sales_Decile"]==n_decile]["Total_of_Sales"].min())

デシル1の最大値: 14802928.0
デシル1の最小値: 6930.0
